<a href="https://colab.research.google.com/github/damianozanardini/nodePrediction/blob/main/NodePrediction_submit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New node prediction

## Damiano Zanardini and Emilio Serrano

This work has been published as [New node prediction: a novel learning task for Graph Neural Networks](https://www.sciencedirect.com/science/article/pii/S0925231224012451?dgcid=coauthor).

This notebook wants to be an implementation of the framework that is flexible in terms of the graph it works on.

## Setup

### Installs and imports

We first will install [PyG](https://pyg.org/) (PyTorch Geometric), together with [ogb](https://github.com/snap-stanford/ogb) (Open Graph Benchmark) and the usual packages.

Due to unknown reasons, to install PyG and related packages on Colab may lead to some unexpected behavior (namely, it may take nearly one hour instead of the usual 30/40 seconds). Because of this, we include a couple of code cells with code for installing PyG that worked properly at times in the past.

In [1]:
# This is the better-working approach so far

drive = False # Should consider executing on Google Drive but also on our cluster, not only local

import torch
import os

os.environ['TORCH'] = torch.__version__
os.environ['TORCH_DOWNPATH'] = "https://pytorch-geometric.com/whl/" # "https://data.pyg.org/whl/"

if drive:
    !pip uninstall torch-scatter torch-geometric --y

    !pip install git+https://github.com/pyg-team/pytorch_geometric.git
    !pip install torch-scatter -f ${TORCH_DOWNPATH}torch-${TORCH}.html
    !pip install torch-sparse -f ${TORCH_DOWNPATH}torch-${TORCH}.html

In [ ]:
################################################################################
################################################################################
# [NOT TO BE RUN NORMALLY] Alternatives for installing PyG (1) #################

import torch
import os

!pip install torch_geometric

# Optional dependencies:
!pip install torch_scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html

In [ ]:
################################################################################
################################################################################
# [NOT TO BE RUN NORMALLY] Alternatives for installing PyG (2) #################

import torch
import os
os.environ['TORCH'] = torch.__version__
print("PyTorch has version {}".format(torch.__version__))

#down_url = "https://data.pyg.org/whl/"
down_url = "https://pytorch-geometric.com/whl/"

# Installing torch-scatter, torch-sparse and torch-geometric is somehow painful
# Sometimes it is necessary to change the download urls
!pip uninstall torch-scatter torch-sparse torch-geometric --y

!pip install torch-scatter -f {down_url}torch-{torch.__version__}.html
!pip install torch-sparse -f {down_url}torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git
#!pip install torch-geometric \
#  torch-sparse \
#  torch-scatter \
#  -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html

#!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-${TORCH}.html
#!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-${TORCH}.html
#!pip install torch-geometric

#!pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
#!pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
#!pip install torch-geometric

In [2]:
if drive:
    !pip install ogb

import torch.nn.functional as F
import networkx as nx
from networkx.generators import random_graphs

import random
import torch_geometric.transforms as T

from torch import Tensor
from torch.utils.data import DataLoader
import torch_geometric.utils as utils
from torch_geometric.nn import GCNConv, ClusterGCNConv, GATConv, SAGEConv
from torch_geometric.nn.conv import MessagePassing
from ogb.linkproppred import PygLinkPropPredDataset, Evaluator

from torch_geometric.data import Data
from torch_geometric.utils import subgraph

import math
from statistics import mean
import calendar
import time
from datetime import datetime

import pandas as pd
from openpyxl import load_workbook

if drive:
    !pip install torchmetrics
from torchmetrics import PearsonCorrCoef

import sys

import import_ipynb

/Users/damiano/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
import ogbl_citation2
import my_utils

importing Jupyter notebook from ogbl_citation2.ipynb
importing Jupyter notebook from my_utils.ipynb
Initializing stored_graph to None
Damiano


### Global vars/options

In [4]:
# GPU is selected whenever available
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    # This is for M-series-based Macs
    if torch.backends.mps.is_available():
        device = torch.device("mps")
    else:
        device = torch.device("cpu")
print(f'Device is {device}')

dir = '/content/drive/MyDrive/Colab Notebooks/NodePrediction/' if drive else os.getcwd()

# The original graph
graph_name = 'ogbl-citation2'

# Given a graph, the proportion of train nodes vs. test nodes
# (80%/20% is the split that is used in the paper)
train_ratio = 0.8

# Whether computing times for the main training steps are shown
get_times = False

Device is mps


### Loading the selected type of graph

In [5]:

def get_graph(graph_name,graph_type,n_nodes,node_features_type):
    graph0 = sys.modules[graph_name].get_graph(n_nodes,node_features_type)

    if graph_type == 'sample':
        graph = graph0
        g_type = "Sampled"
        train_graph, train_nodes, test_nodes = my_utils.get_training_graph(graph)
    elif graph_type == 'sample_undir':
        graph = Data(x=graph0.x,edge_index=utils.to_undirected(graph0.edge_index))
        g_type = "Sampled (unidirected)"
        train_graph, train_nodes, test_nodes = my_utils.get_training_graph(graph)
    elif graph_type == 'ego': # Requires big graph to be loaded
        graph = my_utils.get_ego_network_un(n_nodes)
        g_type = "EGO"
        train_graph, train_nodes, test_nodes = my_utils.get_training_graph(graph)
    elif graph_type == 'ba':
        graph = my_utils.get_barabasi_albert_graph(n_nodes=n_nodes,n_edges=n_edges,num_node_features=num_features)
        g_type = "Barabási-Albert"
        train_graph, train_nodes, test_nodes = my_utils.get_training_graph(graph)
    elif graph_type == 'er':
        graph = my_utils.get_barabasi_albert_graph(n_nodes=n_nodes,n_edges=n_edges,num_node_features=num_features)
        g_type = "Erdös-Renyi"
        train_graph, train_nodes, test_nodes = my_utils.get_training_graph(graph)
    elif graph_type == 'sample+ba':
        graph = my_utils.get_sampled_plus_barabasi_albert_graph(n_nodes,num_features)
        g_type = "Sampled (+Barabási-Albert for test nodes)"
        n_train_nodes = round(n_nodes*train_ratio)
        edges = subgraph(subset=list(range(n_train_nodes)),edge_index=graph.edge_index,relabel_nodes=False)
        print(edges[0].shape)
        train_graph = Data(x=graph.x,edge_index=edges[0])
        nodes = torch.tensor(list(range(n_nodes)))
        train_mask = torch.cat([torch.tensor([True]*n_train_nodes),torch.tensor([False]*(n_nodes-n_train_nodes))])
        train_nodes = nodes[train_mask]
        test_mask = torch.logical_not(train_mask)
        test_nodes = nodes[test_mask]
    else: # The "else" case is like sampled, but a warning message is output
        graph = graph0
        g_type = "NOT SPECIFIED"
        train_graph, train_nodes, test_nodes = my_utils.get_training_graph(graph)

    print(f"{g_type} graph with {graph.num_nodes} nodes and {graph.num_edges} edges loaded")

    print(f'The train graph has {len(train_nodes)} (actual) nodes and {train_graph.num_edges} edges')
    print(f'Number of test nodes: {len(test_nodes)}')

    return graph.to(device), train_graph.to(device), train_nodes.to(device), test_nodes.to(device)

## Neural Model

### GNN architectures

We have GCN, GAT, ClusterGCN and SAGE available.
All architectures are a sequence of GNN layers of a given class, with ReLU between layers, and a configurable dropout.

In [6]:
class GCN(torch.nn.Module):
  def __init__(self, in_channels, hidden_channels, out_channels, num_layers, dropout):
    super().__init__()
    self.convs = torch.nn.ModuleList()
    self.convs.append(GCNConv(in_channels, hidden_channels))
    for _ in range(num_layers - 2):
      self.convs.append(GCNConv(hidden_channels, hidden_channels))
    self.convs.append(GCNConv(hidden_channels, out_channels))

    self.norm = torch.nn.ModuleList()
    for _ in range(num_layers-1):
      self.norm.append(torch.nn.BatchNorm1d(hidden_channels))

    self.dropout = dropout

  def reset_parameters(self):
    for conv in self.convs:
      conv.reset_parameters()

  def forward(self, x, edge_index):
    for conv,norm in zip(self.convs[:-1],self.norm):
      x = conv(x, edge_index)
      x = F.relu(x)
      x = norm(x)
      x = F.dropout(x, p=self.dropout, training=self.training)
    x = self.convs[-1](x, edge_index)
    return x

class ClusterGCN(torch.nn.Module):
  def __init__(self, in_channels, hidden_channels, out_channels, num_layers, dropout):
    super().__init__()
    self.convs = torch.nn.ModuleList()
    self.convs.append(ClusterGCNConv(in_channels, hidden_channels))
    for _ in range(num_layers - 2):
      self.convs.append(ClusterGCNConv(hidden_channels, hidden_channels))
    self.convs.append(ClusterGCNConv(hidden_channels, out_channels))

    self.dropout = dropout

  def reset_parameters(self):
    for conv in self.convs:
      conv.reset_parameters()

  def forward(self, x, edge_index):
    for conv in self.convs[:-1]:
      x = conv(x, edge_index)
      x = F.relu(x)
      x = F.dropout(x, p=self.dropout, training=self.training)
    x = self.convs[-1](x, edge_index)
    return x

class GAT(torch.nn.Module):
  def __init__(self, in_channels, hidden_channels, out_channels, num_layers, dropout):
    super().__init__()

    self.convs = torch.nn.ModuleList()
    self.convs.append(GATConv(in_channels, hidden_channels))
    for _ in range(num_layers - 2):
      self.convs.append(GATConv(hidden_channels, hidden_channels))
    self.convs.append(GATConv(hidden_channels, out_channels))

    self.dropout = dropout

  def reset_parameters(self):
    for conv in self.convs:
      conv.reset_parameters()

  def forward(self, x, edge_index):
    for conv in self.convs[:-1]:
      x = conv(x, edge_index)
      x = F.relu(x)
      x = F.dropout(x, p=self.dropout, training=self.training)
    x = self.convs[-1](x, edge_index)
    return x

class SAGE(torch.nn.Module):
  def __init__(self, in_channels, hidden_channels, out_channels, num_layers, dropout, aggr="add"):
    super(SAGE, self).__init__()

    self.convs = torch.nn.ModuleList()
    self.convs.append(SAGEConv(in_channels, hidden_channels, normalize=True, aggr=aggr))
    for _ in range(num_layers - 2):
      self.convs.append(SAGEConv(hidden_channels, hidden_channels, normalize=True, aggr=aggr))
    self.convs.append(SAGEConv(hidden_channels, out_channels, normalize=True, aggr=aggr))

    self.norm = torch.nn.ModuleList()
    for _ in range(num_layers-1):
      self.norm.append(torch.nn.BatchNorm1d(hidden_channels))

    self.dropout = dropout

  def reset_parameters(self):
    for conv in self.convs:
      conv.reset_parameters()

  def forward(self, x, edge_index):
    for conv,norm in zip(self.convs[:-1],self.norm):
      x = conv(x, edge_index)
      x = F.relu(x)
      x = norm(x)
      running_mean = torch.mean(x, -1) # it can't go into the batch_norm function as a parameter
      running_var = torch.var(x, -1) # it can't go into the batch_norm function as a parameter
      # x = torch.nn.functional.batch_norm(x, running_mean, running_var)
      x = F.dropout(x, p=self.dropout, training=self.training)
    x = self.convs[-1](x, edge_index)
    return x

### Architecture for **predictors**

MLPNodePredictor
  - takes an example (sources,target)
  - computes an embedding **e_sources** that is the point-wise average of the embeddings corresponding to sources
  - takes the embedding **e_target** corresponding to target
  - concatenates **e_sources** with **e_target**
  - applies a DL arquitecture to it

In [7]:
class NodePredictor(torch.nn.Module):
  def __init__(self):
    super(NodePredictor, self).__init__()

  def forward(self, examples, node_embs):
    pass

  def reset_parameters(self):
    pass

class MLPNodePredictor(NodePredictor):
  def __init__(self,embedding_size):
    super(MLPNodePredictor, self).__init__()

    # These numbers should better be powers of 2
    hidden_dimension1 = round(embedding_size/2)
    hidden_dimension2 = round(hidden_dimension1/4)

    self.lin1 = torch.nn.Linear(2*embedding_size, hidden_dimension1)
    self.lin2 = torch.nn.Linear(hidden_dimension1, hidden_dimension2)
    self.lin3 = torch.nn.Linear(hidden_dimension2, 1)

  def forward(self,examples,node_embs):
    (sources,targets) = examples

    # Computing embeddings for examples
    # This has to be done each time because the last-generated node embeddings
    # have to be used
    sources_embs = torch.stack([node_embs[source.t()].mean(0) for source in sources])
    sources_embs = torch.nan_to_num(sources_embs)
    target_embs = torch.stack([node_embs[target] for target in targets])
    x = torch.cat([sources_embs,target_embs], 1)

    x = self.lin1(x)
    x = torch.nn.functional.relu(x)
    x = self.lin2(x)
    x = torch.nn.functional.relu(x)
    x = self.lin3(x)
    out = torch.flatten(x)

    return torch.sigmoid(out)

  def reset_parameters(self):
    self.lin1.reset_parameters()
    self.lin2.reset_parameters()
    self.lin3.reset_parameters()

## Example generation

This code is in charge of generating **positive** and **negative** examples, for both **training** and **test**.

What distinguishes a training example from a test one is the set of nodes the **target node** is taken from: either the train or the test portion of the graph.

In [8]:
# This function takes a set of nodes and returns a randomly generated subset of
# it such that the ratio of selected nodes is between 'at_least' and 'at_most'
# (both taking values between 0 and 1)
def filter_nodes(nodes,at_least,at_most):
    if at_least>at_most: # this should never happen
        at_most = at_least

    l = len(nodes)
    # nodes are first shuffled
    rand_indx = torch.randperm(l)
    nodes = nodes[rand_indx]

    mask = torch.rand(l)
    mask = mask>0.5
    for i in range(l):
        if i<l*at_least: # nodes that DO have to be included
            mask[i] = True
        elif i>=l*at_most: # nodes that DO NOT have to be included
            mask[i] = False

    return nodes[mask]

# This function generates a list of examples of type T with T in {pos,neg}.
# This means that it comes up with randomly chosen pairs (S,t) such that
# - S is a set of nodes
# - t is a node
# - if T is 'pos': a minimal proportion 'min_pure' of nodes s such that (s,t)
#   is in the graph (a positive edge) is in S, and a maximal proportion
#   'max_spurious' of nodes s such that (s,t) is not in the graph (a negative
#   edge) is in S
# - is T is 'neg': the other way around
def generate_examples(type,total_nodes,target_nodes,train_sample_ratio,pos_edges,neg_edges,min_pure=1.0,max_spurious=0.0):
    sources = torch.zeros(len(target_nodes),total_nodes).bool().to(device)
    targets = torch.zeros(len(target_nodes)).int().to(device)

    n = 0
    rand_sample = torch.randperm(len(target_nodes))[:(round(train_sample_ratio*len(target_nodes)))]

    for target in target_nodes[rand_sample]:
        # pick randomly a node target with incoming edges
        #target = target_nodes[random.randint(0,len(target_nodes)-1)]
        # take nodes with POSITIVE outcoming edges pointing to target
        pos_sources = pos_edges[0,pos_edges[1,:]==target]
        # take nodes with NEGATIVE outcoming edges pointing to target
        neg_sources = neg_edges[0,neg_edges[1,:]==target]

        if type == 'pos':
            # positive edges are considered as pure
            filtered_pos_sources = filter_nodes(pos_sources,min_pure,1.00)
            # negative edges are considered as spurious
            filtered_neg_sources = filter_nodes(neg_sources,0.00,max_spurious)
        else:
            # positive edges are considered as spurious
            filtered_pos_sources = filter_nodes(pos_sources,0.00,max_spurious)
            # negative edges are considered as pure
            filtered_neg_sources = filter_nodes(neg_sources,min_pure,1.00)

        # this loop is for debug purposes only, when torch_scatter is nos imported
        # for x in torch.cat([filtered_pos_sources,filtered_neg_sources]):
        # sources[n][x] = True
        sources[n].scatter_(dim=0, index=torch.cat([filtered_pos_sources,filtered_neg_sources]), value=True)
        targets[n] = int(target)

        n = n+1

    return (sources,targets)

# This function is supposed to generate labeled positive and negative examples
# related to target nodes that are in a random set nodes of nodes.
#
# One positive and one negative example is generated for each target node
def create_labeled_examples(total_nodes,target_nodes,edge_index,neg_edges,purity,train_sample_ratio=1.0):
    n_pos_examples = n_neg_examples = round(train_sample_ratio*len(target_nodes))
    # Positive examples are generated such as they have a node from 'nodes' as their target
    (pos_sources,pos_targets) = generate_examples('pos',
                                                total_nodes=total_nodes,
                                                target_nodes=target_nodes,
                                                train_sample_ratio=train_sample_ratio,
                                                pos_edges=edge_index,
                                                neg_edges=neg_edges,
                                                min_pure = purity['min_pure'],
                                                max_spurious = purity['max_spurious'])
    pos_labels = torch.ones(n_pos_examples).to(device)
    # Negative examples are generated such as they have a node from 'nodes' as their target
    (neg_sources,neg_targets) = generate_examples('neg',
                                                total_nodes=total_nodes,
                                                target_nodes=target_nodes,
                                                train_sample_ratio=train_sample_ratio,
                                                pos_edges=edge_index,
                                                neg_edges=neg_edges,
                                                min_pure = purity['min_pure'],
                                                max_spurious = purity['max_spurious'])
    neg_labels = torch.zeros(n_neg_examples).to(device)

    # Joining positive and negative examples and shuffling
    train_sources = torch.cat([pos_sources,neg_sources]).to(device)
    train_targets = torch.cat([pos_targets,neg_targets]).to(device)
    train_labels = torch.cat([pos_labels,neg_labels]).to(device)
    rp = torch.randperm(n_pos_examples+n_neg_examples)
    return (train_sources[rp],train_targets[rp]), train_labels[rp]

## Training driver

The GNN runs on the train part of the graph (a complete subgraph with a portion of nodes, and the corresponding edges between them).

In [9]:
# Patience is the number of epochs after which we want to stop on no improvements
def early_stopping(epoch,model,predictor,h,val_examples,val_labels,val_loss_history,best_val_loss,patience):
    val_preds = predictor(val_examples,h)
    val_correct_predictions = len([n for n in val_preds[val_labels==1.0] if n>=0.5]) + len([n for n in val_preds[val_labels==0.0] if n<0.5])
    val_loss = loss_fn(val_preds, val_labels)
    val_loss_history.append(val_loss.item())
    print(f"Computing validation loss: {val_loss.item():.4f}")
    if val_loss<best_val_loss:
        print(f"Saving best model (epoch {epoch})")
        torch.save(model.state_dict(),dir + "best_model.pth")
        torch.save(predictor.state_dict(),dir+ "best_predictor.pth")
        best_val_loss = val_loss
    if len(val_loss_history)>patience:
        x = val_loss_history[-(patience+1)]
        es = True
        for i in range(patience):
            es = es & (x<=val_loss_history[-(patience-i)])
        if es:
            print(f"Val loss {x:.4f} at epoch {len(val_loss_history)-patience} better than {val_loss_history[-patience:]}")
    else:
        es = False
    return (es,best_val_loss)

def train(model,predictor,train_graph,train_nodes,train_sample_ratio,loss_fn,optimizer,batch_size,num_epochs,purity):
    check_early_stopping = True

    # Relative size of the validation set (only used with early stopping)
    val_size = 0.1 if check_early_stopping else 0.0
    best_val_loss = float('inf')

    model.train()
    predictor.train()

    # Generating negative edges (once and for all)
    neg_edges = utils.negative_sampling(train_graph.edge_index, num_nodes=len(train_graph.x),
                                num_neg_samples=train_graph.edge_index.shape[1], method='dense').to(device)
    # At training time, negative edges have to be restricted to the train graph
    # Due to this, and to how negative edges are created, the number of negative
    # edges comes to be a bit smaller than positive edges.
    neg_edges = subgraph(subset=train_nodes,edge_index=neg_edges,relabel_nodes=False)[0]

    # Keeping track of loss history, together with the best epoch for loss and accuracy
    loss_history = []
    val_loss_history = []
    # To be returned later
    # Minimum loss
    min_loss = float('inf')
    # Epoch at which the minimum loss is reached
    min_loss_epoch = 0
    # Maximum accuracy
    max_acc = 0.0
    # Epoch at which the maximum accuracy is reached
    max_acc_epoch = 0

    n_pos_examples = n_neg_examples = round(train_sample_ratio*len(train_nodes))

    start_time = time.time() if get_times else 0
    print(f"Generating {n_pos_examples}+{n_neg_examples} total examples")
    (sources,targets), labels = create_labeled_examples(total_nodes=train_graph.num_nodes,
                                                      target_nodes=train_nodes,
                                                      train_sample_ratio=train_sample_ratio,
                                                      edge_index=train_graph.edge_index,
                                                      neg_edges=neg_edges,
                                                      purity=purity)

    # If needed, splitting into train and validation examples
    limit = round(len(labels)*(1-val_size))
    train_sources = sources[:limit]
    train_targets = targets[:limit]
    train_examples = (train_sources,train_targets)
    train_labels = labels[:limit]
    val_sources = sources[limit:]
    val_targets = targets[limit:]
    val_examples = (val_sources,val_targets)
    val_labels = labels[limit:]
    print(f"{labels.shape[0]} examples generated: {train_labels.shape[0]} train examples + {val_labels.shape[0]} val examples")
    my_utils.print_time("Example-generation",start_time)

    print(f"Positive labels: {train_labels.sum(-1)}")

    for epoch in range(1,num_epochs+1):
        epoch_start_time = time.time() if get_times else 0
        epoch_correct_predictions = 0
        epoch_total_predictions = 0
        epoch_total_loss = 0

        for perm in DataLoader(range(train_examples[0].shape[0]), batch_size, shuffle=True):
            batch_start_time = time.time() if get_times else 0
            optimizer.zero_grad()

            batch_examples = (train_sources[perm],train_targets[perm])
            batch_labels = train_labels[perm]

            # Using the GNN to generate node embeddings
            # Embeddings are computed for ALL the training graph every time
            start_time = time.time() if get_times else 0
            h = model(train_graph.x, train_graph.edge_index)
            my_utils.print_time("Node-embeddings",start_time)

            # Getting predictions for our batch, and computing the training loss
            start_time = time.time() if get_times else 0
            preds = predictor(batch_examples,h)
            epoch_correct_predictions += len([n for n in preds[batch_labels==1.0] if n>=0.5]) + len([n for n in preds[batch_labels==0.0] if n<0.5])
            epoch_total_predictions += preds.shape[0]
            loss = loss_fn(preds, batch_labels)
            epoch_total_loss += loss.item()
            my_utils.print_time("Prediction",start_time)

            # Updating our parameters
            start_time = time.time() if get_times else 0
            loss.backward()
            my_utils.print_time("Backpropagation",start_time)
            start_time = time.time() if get_times else 0
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            torch.nn.utils.clip_grad_norm_(predictor.parameters(), 1.0)
            optimizer.step()
            my_utils.print_time("Optimization",start_time)
            my_utils.print_time("Batch",batch_start_time)

        epoch_acc = 100*epoch_correct_predictions/epoch_total_predictions

        # Updating loss and accuracy history
        loss_history.append(epoch_total_loss)
        if epoch_total_loss < min_loss:
            min_loss = epoch_total_loss
            min_loss_epoch = epoch
        if epoch_acc > max_acc:
            max_acc = epoch_acc
            max_acc_epoch = epoch

        if check_early_stopping:
            patience = 8
            es,bvs = early_stopping(epoch=epoch,
                              model=model,
                              predictor=predictor,
                              h=h,
                              val_examples=val_examples,
                              val_labels=val_labels,
                              val_loss_history=val_loss_history,
                              best_val_loss=best_val_loss,
                              patience=patience)
            best_val_loss = bvs # updating best_val_loss
            if es:
                print(f'EARLY STOPPING AT EPOCH {epoch}; best model: epoch {epoch-patience}')
                break

        torch.save(model.state_dict(),dir + f"model_{epoch}.pth")
        torch.save(predictor.state_dict(),dir+ f"predictor_{epoch}.pth")

        print(f'Epoch {(epoch):4d} has loss {round(epoch_total_loss, 4):.4f}; ' +
          f'correct predictions: {epoch_correct_predictions:6d} out of {epoch_total_predictions:6d} ' +
          f'({epoch_acc:3.4f}%)')

        my_utils.print_time("Epoch",epoch_start_time)

    return min_loss, min_loss_epoch, max_acc, max_acc_epoch

## Test driver

This code
- generates embeddings for previously unseen nodes (not belonging to the train graph)
- generates positive and negative examples with those nodes as target
- classifies the examples
- returns several metrics: accuracy, Hits@K, MRR

In [10]:
def get_accuracy(loss,test_correct_predictions,test_total_predictions):
    acc = 100*test_correct_predictions/test_total_predictions
    return acc

def get_hits_at_k(k, test_examples, test_labels, preds):
    pos_examples_preds = preds[test_labels==1.0]
    n_pos = len(pos_examples_preds)
    neg_examples_preds = preds[test_labels==0.0]
    n_neg = len(neg_examples_preds)

    hits = torch.tensor([len((neg_examples_preds>p).nonzero()) for p in pos_examples_preds])
    h = len((hits<k).nonzero())/n_pos
    return h

def get_mrr(test_examples, test_labels, preds):
    pos_examples_preds = preds[test_labels==1.0]
    n_pos = len(pos_examples_preds)
    neg_examples_preds = preds[test_labels==0.0]
    n_neg = len(neg_examples_preds)

    ranks = torch.tensor([len((neg_examples_preds>p).nonzero())+1 for p in pos_examples_preds])
    ranks_reciprocal = torch.reciprocal(ranks)

    mrr = (ranks_reciprocal.sum())/n_pos
    return mrr.item()

def test(model,predictor,test_graph,test_nodes,loss_fn,num_test_examples,purity):
    num_nodes = len(test_graph.x)

    # Generating negative edges (once and for all)
    neg_edges = utils.negative_sampling(test_graph.edge_index, num_nodes=num_nodes,
                                num_neg_samples=test_graph.edge_index.shape[1], method='dense').to(device)

    optimizer.zero_grad()

    test_examples, test_labels = create_labeled_examples(total_nodes=test_graph.num_nodes,
                                                       target_nodes=test_nodes,
                                                       edge_index=test_graph.edge_index,
                                                       neg_edges=neg_edges,
                                                       purity=purity)

    # Use the GNN to generate node embeddings
    h = model(test_graph.x, test_graph.edge_index)
    # print(f'Model output: {h} (shape {h.shape})')

    # Get predictions and compute the loss
    preds = predictor(test_examples,h)
    test_correct_predictions = len([n for n in preds[test_labels==1.0] if n>=0.5]) + len([n for n in preds[test_labels==0.0] if n<0.5])
    test_total_predictions = preds.shape[0]

    loss = loss_fn(preds, test_labels)

    # Printing results
    accuracy = get_accuracy(loss,test_correct_predictions,test_total_predictions)
    print(f'Test loss: {round(loss.item(), 4)}; ' +
          f'Accuracy (correct predictions): {test_correct_predictions} out of {test_total_predictions} ' +
          f'({accuracy:.4f}%)')
    hits = {}
    num_neg_examples = len(preds[test_labels==0.0])
    print(f"Hits@k (wrt {num_neg_examples} negative examples): ", end="")
    for k in [1,2,3,5,10,20,30,50]:
        hits[k] = get_hits_at_k(k, test_examples, test_labels, preds)
        print(f"k={k}: {hits[k]:.4f}; ", end="")
    print()
    mrr = get_mrr(test_examples, test_labels, preds)
    print(f"MRR: {mrr:.4f}")

    return accuracy, hits, mrr

## Training and testing the models

### Run configurations

The **options** dictionary assigns values for different parameters.
The following cell runs tests corresponding to such parameters.

For each parameter, a list of values can be specified; if the list contains more than one value, then differnt test will be run.

Ex. `options['sampled_nodes'] = [3000,10000]` runs experiments on the 3000-nodes and 10000-nodes of a graph.

Moreover, it is possible to specify the value(s) of a parameter as dependent on some other parameter: for example, tha size of a batch may depend on the size of the graph. This can be done by assigning a dictionary, instead of a list, to a parameter.

Ex. `options['batch_size'] = { 3000: [512], 10000: [2048], 20000: [4096], 50000: [8192] }` specifies a different value for the batch size depending on the size of the graph.

In [ ]:
graph_name = 'ogbl-citation2'

options = {}

# The type of graph
options['graph_type'] = ['sample_undir'] #['sample','sample_undir','ego','ba','er','sample+ba']
# The number of total nodes (in some graph types, like ego, the number of final node is normally bigger)
# If the selected grph is smaller than this number, then it is taken as it is; otherwise, a sample is loaded or computed
options['n_nodes'] = [3000]
# Whether we use the original 128-dimensional vector of node features, or we compute stats-related ones, or we use dummy ones [1]
options['node_features'] = ['original'] #['original','stats','dummy']
# Type of GNN architecture
options['GNN'] = [SAGE]
# Number of layers (it should probably depend on 'sampled_nodes' and/or 'GNN')
options['num_layers'] = { GCN: [5], ClusterGCN: [5], GAT: [5], SAGE: [5] }
# Dropout coefficient (it should probably depend on 'GNN' and/or 'num_layers')
options['dropout'] = { GCN: [0.2], ClusterGCN: [0.2], GAT: [0.2], SAGE: [0.2] }
# Size of the embeddings vector computed by the GNN
options['embedding_size'] = { GCN: [128], ClusterGCN: [128], GAT: [128], SAGE: [128] }
# degree of purity (how many pure edges has to be included in an example, and how many spurious edges are allowed)
# Let G be the graph under study, and G- be the negative graph (i.e., the graph
# with the same nodes that is made of all the negative edges).

# Given an example x=(S,t), that can be a positive or negative one, an edge e is
# pure for x if both e and x have the same polarity (i.e., both positive or both
# negative); otherwise, it is spurious

# This parameter indicates how much "impurity" is allowed in an example:
# Let t be the target node of the example; then, let S be the set of nodes in G
# such that, for every s in S, the edge (s,t) is in G; let also S- be the set of
# nodes in G- such that, for every s in S, the edge (s,t) is in G-. In other
# words, given t, S (resp., S-) are the nodes connected to t by positive (resp.,
# negative) edges.
# The parameter min_pure indicates either
# - the MINIMUM proportion of S that has to be included in a positive example on t; or
# - the MINIMUM proportion of S- that has to be included in a negative example on t
# The parameter max_spurious indicates either
# - the MAXIMUM proportion of S that has to be included in a negative example on t; or
# - the MAXIMUM proportion of S- that has to be included in a positive example on t
#
# For example, if we are considering a positive example, S contains 20 nodes, S-
# contains 10 nodes, min_pure=0.8, max_spurious=0.2: the example will contain
# AT LEAST 16=20*0.8 positive edges and AT MOST 2=10*0.2 negative edges
#
# Purity may be different at train and test time
options['train_purity'] = [{'min_pure': 0.8,'max_spurious': 0.1}] # [{'min_pure': 1.0, 'max_spurious': 0.0},{'min_pure': 0.8,'max_spurious': 0.1},{'min_pure': 0.5,'max_spurious': 0.2}]
# In general, one positive and one negative example is generated for every target node at training time; however, it is possible to specify a ratio of nodes for which examples are generated
options['train_sample_ratio'] = [1.0]#,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
options['learning_rate'] = { GCN: [0.001,0.01,0.05], ClusterGCN: [0.001,0.01,0.05], GAT: [0.001,0.01,0.05], SAGE: [0.001] }
options['test_purity'] = [{'min_pure': 0.8,'max_spurious': 0.1}] # [{'min_pure': 1.0, 'max_spurious': 0.0},{'min_pure': 0.8,'max_spurious': 0.1},{'min_pure': 0.5,'max_spurious': 0.2}]
# the size of each training batch (it depends on 'sampled_nodes')
options['batch_size'] = { 3000: [512], 10000: [2048], 20000: [4096], 50000: [8192] }
# the number of training epochs
options['num_epochs'] = [5]

number_of_runs = 1 # This is used to run the same test several times

n = 0
for graph_type in options['graph_type']:
  for n_nodes in options['n_nodes']:
    for node_features in options['node_features']:
      for gnn in options['GNN']:
        for num_layers in options['num_layers'][gnn]:
          for embedding_size in options['embedding_size'][gnn]:
            for dropout in options['dropout'][gnn]:
              for batch_size in options['batch_size'][sample_nodes]:
                for num_epochs in options["num_epochs"]:
                  for train_purity in options['train_purity']:
                    for train_sample_ratio in options['train_sample_ratio']:
                      for learning_rate in options['learning_rate'][gnn]:
                        for test_purity in options['test_purity']:
                          n = n+1
print(f'A total number of {n} learning tasks will be executed {number_of_runs} times')


## Executing the battery of runs

Experiments are run based on the values specified in the options dictionary.

In [ ]:
n = 0
loss_fn = torch.nn.BCELoss()

this_run = 0
while (this_run<number_of_runs):
  for graph_type in options['graph_type']:
    for n_nodes in options['n_nodes']:
      for node_features in options['node_features']:
        print("### DATA")
        graph, train_graph, train_nodes, test_nodes = get_graph(graph_name,graph_type,n_nodes,node_features)
        num_features = len(graph.x[0])
        for gnn in options['GNN']:
          for num_layers in options['num_layers'][gnn]:
            for embedding_size in options['embedding_size'][gnn]:
              for dropout in options['dropout'][gnn]:
                # Defining the neural architecture
                hidden_dimension = embedding_size
                print(f"### MODEL: {gnn.__name__} with {num_layers} layers, {num_features} inputs, {hidden_dimension} hidden dimensions, and dropout {dropout}")
                # Doing the proper training
                for batch_size in options['batch_size'][n_nodes]:
                  for num_epochs in options["num_epochs"]:
                    for train_purity in options['train_purity']:
                      for train_sample_ratio in options['train_sample_ratio']:
                        for learning_rate in options['learning_rate'][gnn]:
                          num_train_examples = round(train_sample_ratio*len(train_nodes))
                          model = gnn(num_features,
                                      hidden_dimension,
                                      hidden_dimension,
                                      num_layers=num_layers,
                                      dropout=dropout).to(device)
                          predictor = MLPNodePredictor(embedding_size=embedding_size).to(device)
                          print(model)
                          print(predictor)
                          # Preparing training
                          optimizer = torch.optim.Adam(list(model.parameters()) + list(predictor.parameters()), lr=learning_rate)
                          print(f"### TRAINING with {num_train_examples}+{num_train_examples} ({(100*train_sample_ratio):1f}% sample) train/val examples, purity ({train_purity['min_pure']}, {train_purity['max_spurious']}), batch_size = {batch_size}, num_epochs = {num_epochs}, learning rate = {learning_rate}")
                          start_time = time.time()
                          min_loss, min_loss_epoch, max_acc, max_acc_epoch = train(model=model,
                                                                                   predictor=predictor,
                                                                                   train_graph=train_graph,
                                                                                   train_nodes=train_nodes,
                                                                                   train_sample_ratio=train_sample_ratio,
                                                                                   loss_fn=loss_fn,
                                                                                   optimizer=optimizer,
                                                                                   batch_size=batch_size,
                                                                                   num_epochs=num_epochs,
                                                                                   purity=train_purity)
                          end_time = time.time()
                          train_time = end_time-start_time
                          print(f"Minimum loss:     {min_loss:.4f} reached at epoch {min_loss_epoch}")
                          print(f"Maximum accuracy: {max_acc:.4f} reached at epoch {max_acc_epoch}")
                          print(f"Training time: {train_time} s")
                          # Testing the results
                          num_test_examples = len(test_nodes)
                          for test_purity in options['test_purity']:
                            n_tests = 5
                            print(f"\n### TEST (x{n_tests}) with {num_test_examples}+{num_test_examples} examples and purity ({test_purity['min_pure']}, {test_purity['max_spurious']})")
                            accuracy = [0]*n_tests
                            hits = [0]*n_tests
                            mrr = [0]*n_tests
                            # Loading best model and predictor
                            model.load_state_dict(torch.load(dir + 'best_model.pth'))
                            predictor.load_state_dict(torch.load(dir + 'best_predictor.pth'))
                            for i in range(n_tests):
                              accuracy[i], hits[i], mrr[i] = test(model=model,
                                                                  predictor=predictor,
                                                                  test_graph=graph,
                                                                  test_nodes=test_nodes,
                                                                  loss_fn=loss_fn,
                                                                  num_test_examples=num_test_examples,
                                                                  purity=test_purity)
                            accuracy_mean = mean(accuracy)
                            # neg_sources = neg_edges[0,neg_edges[1,:]==target]
                            hits_mean = {}
                            for k in hits[0].keys():
                              hits_mean[k] = mean([hits[i][k] for i in range(n_tests)])
                            mrr_mean = mean(mrr)

                            print(f"\nAverage values (on {n_tests} test runs)")
                            print(f"- Accuracy (correct predictions): {accuracy_mean:.4f}%")
                            print(f"- Hits@k (wrt {num_test_examples} negative examples): ", end="")
                            for k in hits[0].keys():
                              print(f"k={k}: {hits_mean[k]:.4f}; ", end="")
                            print()
                            print(f"- MRR: {mrr_mean:.4f}")

                            output = {
                              "graph_type": graph_type,
                              "graph_nodes": graph.num_nodes,
                              "node_features": node_features,
                              "gnn": gnn.__name__,
                              "num_layers": num_layers,
                              "embedding_size": embedding_size,
                              "dropout": dropout,
                              "num_train_examples": num_train_examples,
                              "train_purity": f"({train_purity['min_pure']}, {train_purity['max_spurious']})",
                              "batch_size": batch_size,
                              "learning_rate": learning_rate,
                              "num_epochs": num_epochs,
                              "min_loss": min_loss,
                              "min_loss_epoch": min_loss_epoch,
                              "max_acc": max_acc,
                              "max_acc_epoch": max_acc_epoch,
                              "train_time": train_time,
                              "num_test_examples": num_test_examples,
                              "test_purity": f"({test_purity['min_pure']}, {test_purity['max_spurious']})",
                              "accuracy_mean": accuracy_mean,
                              "num_neg_examples": num_test_examples,
                              "hits_mean1": hits_mean[1],
                              "hits_mean2": hits_mean[2],
                              "hits_mean3": hits_mean[3],
                              "hits_mean5": hits_mean[5],
                              "hits_mean10": hits_mean[10],
                              "hits_mean20": hits_mean[20],
                              "hits_mean30": hits_mean[30],
                              "hits_mean50": hits_mean[50],
                              "mrr_mean": mrr_mean
                            }

                            write_line_to_xlsx(output)
                            print(f"### Learning task #{n+1} done\n\n")
                            n = n+1
  this_run = this_run+1
print(f'Executed {n} learning tasks')

# Scratch

In [6]:
%run "./my_utils.ipynb"
%run "./ogbl_citation2.ipynb"

Initializing stored_graph to None
Damiano


In [7]:
g = ogbl_citation2.get_graph(dir,3000,'original')

UnboundLocalError: local variable 'stored_graph' referenced before assignment